# Scaling Elastic Wave Equations from Virieux 1986

In [2]:
# import dependencies
import devito as dv
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from examples.seismic.source import TimeAxis
from examples.seismic import RickerSource

### Implementing unscaled constant parameter propagation

In [38]:
# set up grid
extent = (2000, 2000)
shape = (201, 201)
x = dv.SpaceDimension('x')
z = dv.SpaceDimension('z')
grid = dv.Grid(extent=extent, shape=shape, dimensions=(x, z))

# create the velocity and pressure fields
v = dv.VectorTimeFunction(name='v', grid=grid, space_order=8, time_order=1)
tau = dv.TensorTimeFunction(name='tau', grid=grid, space_order=8, time_order=1)

# define constants
constants = {
    "vp": 3,
    "vs": 1.5,
    "density": 2.4,
    "f0": 0.03
}

# find dt using CFL condition
vmax = max(constants['vp'], constants['vs'])
dt = np.amin(grid.spacing)*0.6/vmax

# define a time range for the source term
t0 = 0
tn = 250
time_range = TimeAxis(start=t0, stop=tn, step=dt)

# define equations
eq_vx = v[0].dt - 1/constants['density']*(tau[0, 0].dx + tau[0, 1].dz)
eq_vz = v[1].dt - 1/constants['density']*(tau[0, 1].dx + tau[1, 1].dz)
eq_sxx = tau[0, 0].dt - constants['vp']**2*constants['density']*v[0].dx + constants['density']*(constants['vp']**2 - 2*constants['vs']**2)*v[1].dz
eq_szz = tau[1, 1].dt - constants['vp']**2*constants['density']*v[1].dz + constants['density']*(constants['vp']**2 - 2*constants['vs']**2)*v[0].dx
eq_sxz = tau[0, 1].dt - constants['vs']**2*constants['density']*(v[0].dz + v[1].dx)